## Подзапросы

**Задача 1.** Используя данные из таблицы user_actions, рассчитайте среднее число заказов всех пользователей нашего сервиса.

Для этого сначала в подзапросе посчитайте, сколько заказов сделал каждый пользователь, а затем обратитесь к результату подзапроса в блоке FROM и уже в основном запросе усредните количество заказов по всем пользователям.

Полученное среднее число заказов всех пользователей округлите до двух знаков после запятой. Колонку с этим значением назовите orders_avg.

In [ ]:
SELECT round(avg (cnt), 2) as orders_avg
FROM   (SELECT user_id,
               count (order_id) as cnt
        FROM   user_actions
        WHERE  action = 'create_order'
        GROUP BY user_id) as sbq_1

**Задача 2.** Повторите запрос из предыдущего задания, но теперь вместо подзапроса используйте оператор WITH и табличное выражение.

Условия задачи те же: используя данные из таблицы user_actions, рассчитайте среднее число заказов всех пользователей.

Полученное среднее число заказов округлите до двух знаков после запятой. Колонку с этим значением назовите orders_avg.

In [ ]:
with sbq_1 as (SELECT user_id,
                      count (order_id) as cnt
               FROM   user_actions
               WHERE  action = 'create_order'
               GROUP BY user_id)
SELECT round(avg (cnt), 2) as orders_avg
FROM   sbq_1

**Задача 3.** Выведите из таблицы products информацию о всех товарах кроме самого дешёвого. Результат отсортируйте по убыванию id товара.

In [ ]:
SELECT product_id,
       name,
       price
FROM   products
WHERE  price != (SELECT min (price)
                 FROM   products)
ORDER BY 1 desc

**Задача 4.** Выведите информацию о товарах в таблице products, цена на которые превышает среднюю цену всех товаров на 20 рублей и более. Результат отсортируйте по убыванию id товара.

In [ ]:
with sbq as (SELECT avg (price)
             FROM   products)
SELECT product_id,
       name,
       price
FROM   products
WHERE  price >= (SELECT *
                 FROM   sbq)+20
ORDER BY 1 desc

**Задача 5.** Посчитайте количество уникальных клиентов в таблице user_actions, сделавших за последнюю неделю хотя бы один заказ. Полученную колонку с числом клиентов назовите users_count. В качестве текущей даты, от которой откладывать неделю, используйте последнюю дату в той же таблице user_actions.

In [ ]:
SELECT count (distinct user_id) as users_count
FROM   user_actions
WHERE  order_id >= 1
   and time >= (SELECT max(time)-interval '1 week'
             FROM   user_actions) and action = 'create_order'

**Задача 6.** С помощью функции AGE и агрегирующей функции снова определите возраст самого молодого курьера мужского пола в таблице couriers, но в этот раз при расчётах в качестве первой даты используйте последнюю дату из таблицы courier_actions.

Чтобы получить именно дату, перед применением функции AGE переведите последнюю дату из таблицы courier_actions в формат DATE, как мы делали в этом задании.

Возраст курьера измерьте количеством лет, месяцев и дней и переведите его в тип VARCHAR. Полученную колонку со значением возраста назовите min_age.

In [ ]:
SELECT min(age((SELECT max(time)
                FROM   courier_actions)::date, birth_date))::varchar as min_age
FROM   couriers
WHERE  sex = 'male'

**Задача 7.** Из таблицы user_actions с помощью подзапроса или табличного выражения отберите все заказы, которые не были отменены пользователями.

Выведите колонку с id этих заказов. Результат запроса отсортируйте по возрастанию id заказа.

Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

In [ ]:
SELECT order_id
FROM   user_actions
WHERE  order_id not in (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')
ORDER BY 1 limit 1000

**Задача 8.** Используя данные из таблицы user_actions, рассчитайте, сколько заказов сделал каждый пользователь и отразите это в столбце orders_count.

В отдельном столбце orders_avg напротив каждого пользователя укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой.

Также для каждого пользователя посчитайте отклонение числа заказов от среднего значения. Отклонение считайте так: число заказов «минус» округлённое среднее значение. Колонку с отклонением назовите orders_diff.

Результат отсортируйте по возрастанию id пользователя. Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

In [ ]:
with sbq as (SELECT user_id,
                    count (order_id) as cnt
             FROM   user_actions
             WHERE  action = 'create_order'
             GROUP BY user_id)
SELECT user_id,
       count (order_id) as orders_count,
       (SELECT round(avg (cnt), 2)
 FROM   sbq) as orders_avg, count (order_id)-(SELECT round(avg (cnt), 2)
                                             FROM   sbq) as orders_diff
FROM   user_actions
WHERE  action = 'create_order'
GROUP BY user_id
ORDER BY 1 limit 1000

**Задача 9.** Назначьте скидку 15% на товары, цена которых превышает среднюю цену на все товары на 50 и более рублей, а также скидку 10% на товары, цена которых ниже средней на 50 и более рублей. Цену остальных товаров внутри диапазона (среднее - 50; среднее + 50) оставьте без изменений. При расчёте средней цены, округлите её до двух знаков после запятой.

Выведите информацию о всех товарах с указанием старой и новой цены. Колонку с новой ценой назовите new_price.

Результат отсортируйте сначала по убыванию прежней цены в колонке price, затем по возрастанию id товара.

In [ ]:
with sbq as (SELECT round(avg(price), 2) as avg_price
             FROM   products)
SELECT product_id,
       name,
       price,
       case when price >= (SELECT avg_price
                    FROM   sbq)+50 then price*0.85 when price <= (SELECT avg_price
                                              FROM   sbq)-50 then price*0.9 else price*1 end as new_price
FROM   products
ORDER BY 3 desc, 1

**Задача 10.** Выясните, есть ли в таблице courier_actions такие заказы, которые были приняты курьерами, но не были созданы пользователями. Посчитайте количество таких заказов. Колонку с числом заказов назовите orders_count.

In [ ]:
SELECT count(order_id) as orders_count
FROM   courier_actions
WHERE  action = 'accept_order'
   and order_id not in (SELECT order_id
                     FROM   user_actions
                     WHERE  action = 'create_order')

**Задача 11.** Выясните, есть ли в таблице user_actions такие заказы, которые были приняты курьерами, но не были доставлены пользователям: у заказа нет записи с действием 'deliver_order' в таблице courier_actions. Посчитайте уникальное количество таких заказов: используйте count(distinct order_id). Колонку с числом заказов назовите orders_count.

In [ ]:
SELECT count(distinct order_id) as orders_count
FROM   user_actions
WHERE  order_id in (SELECT order_id
                    FROM   courier_actions
                    WHERE  action = 'accept_order')
   and order_id not in (SELECT order_id
                     FROM   courier_actions
                     WHERE  action = 'deliver_order')

**Задача 12.** Определите количество отменённых заказов в таблице courier_actions и выясните, есть ли в этой таблице такие заказы, которые были отменены пользователями, но при этом всё равно были доставлены. Посчитайте количество таких заказов. Колонку с отменёнными заказами назовите orders_canceled. Колонку с отменёнными, но доставленными заказами назовите orders_canceled_and_delivered. 

In [ ]:
with sbq as (SELECT order_id
             FROM   user_actions
             WHERE  action = 'cancel_order')
SELECT count(distinct order_id) filter (WHERE order_id in (SELECT *
                                                           FROM   sbq)) as orders_canceled, count(distinct order_id) filter (
WHERE  order_id in (SELECT *
                    FROM   sbq) and action = 'deliver_order') as orders_canceled_and_delivered
FROM   courier_actions